In [1]:
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
import torch
import pymorphy2
import re
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from sklearn.metrics import f1_score, precision_score, recall_score
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
import time
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import CatBoostClassifier
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import seaborn as sns
import pandas as pd
import numpy as np
import re
import os
import random
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import KFold
from catboost import CatBoostClassifier
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
np.random.seed(42)
random.seed(42)
%matplotlib inline

In [2]:
df = pd.read_csv('df_clean.csv')

In [3]:
df.head()

,address,name,rating,rubrics,text,aspects,target,word_pre
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",Московский квартал,3.0,Жилой комплекс,Московский квартал 2.\nШумно : летом по ночам ...,квартал лето ночь гонка стройка окно этаж райо...,1,московский квартал \nшумно лето ночь дикий гон...
1,"Московская область, Электросталь, проспект Лен...",Продукты Ермолино,5.0,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ...",сеть магазин ассортимент цена главное качество...,2,замечательный сеть магазин общий хороший ассор...
2,"Краснодар, Прикубанский внутригородской округ,...",LimeFit,1.0,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я...",правило шкаф замочка отпечаток палец дичь подп...,0,знать смутить дать правило удивить хотеть твой...
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,4.0,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. \nДружелюбный персонал...,условие аренда персонал ботинок крепление сноу...,2,хороший условие аренда \nдружелюбный персонал ...
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,5.0,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...,топ топ смысл посещение бровь ресница итог раб...,2,мастер ангелина весь смысл немного волноваться...


In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("blanchefort/rubert-base-cased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("blanchefort/rubert-base-cased-sentiment", num_labels = 3)

In [7]:
df['text_with_rubric'] = df['rubrics'] + ': ' + df['text']
df = df.dropna(subset=['text_with_rubric'])
def cleann_text(row):
    return re.sub(r'\\|n', '', row)


df['text_with_rubric'] = df['text_with_rubric'].apply(lambda x: cleann_text(x))
df_data = df[['text_with_rubric', 'target']]
data = df_data.rename(columns={"text_with_rubric": "text"})

In [8]:
sentences = data['text'].values
labels = data['target'].values
labels.shape

(118221,)

In [9]:
# Print the original sentence.
print(' Original: ', sentences[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  Жилой комплекс: Московский квартал 2.Шумно : летом по ночам дикие гонки. Грязно : кругом стройки, невозможно открыть окна (16 этаж! ), вечно по району летает мусор. Детские площадки убогие, на большой площади однотипные конструкции. Очень дорогая коммуналка. Часто срабатывает пожарная сигнализация. Жильцы уже не реагируют. В это время, обычно около часа, не работают лифты. Из плюсов - отличная планировка квартир ( Московская 194 ), на мой взгляд. Ремонт от застройщика на 3-. Окна вообще жуть - вместо вентиляции. По соотношению цена/качество - 3.
Tokenized:  ['Жил', '##ой', 'комплекс', ':', 'Московский', 'квартал', '2', '.', 'Шум', '##но', ':', 'летом', 'по', 'ночам', 'дикие', 'гонки', '.', 'Гряз', '##но', ':', 'кругом', 'стройки', ',', 'невозможно', 'открыть', 'окна', '(', '16', 'этаж', '!', ')', ',', 'вечно', 'по', 'району', 'летает', 'мусор', '.', 'Детские', 'площадки', 'убогие', ',', 'на', 'большой', 'площади', 'однотип', '##ные', 'конструкции', '.', 'Очень', 'дорогая', 

In [2]:
input_ids = []
attention_masks = []

for sent in tqdm(sentences):
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, 
                        max_length = 512,           
                        pad_to_max_length = True,
                        return_attention_mask = True,  
                        return_tensors = 'pt',     
                   )

    input_ids.append(encoded_dict['input_ids'])

    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

In [15]:
input_ids.shape

torch.Size([118221, 512])

In [16]:
dataset = TensorDataset(input_ids, attention_masks, labels)

train_size = int(0.75 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

88,665 training samples
29,556 validation samples


In [17]:
batch_size = 16

train_dataloader = DataLoader(
            train_dataset, 
            sampler = RandomSampler(train_dataset), 
            batch_size = batch_size 
        )

validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), 
            batch_size = batch_size 
        )

In [18]:
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [1]:
optimizer = AdamW(model.parameters())

In [20]:
epochs = 2
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_steps)

In [21]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1)
    f1 = f1_score(labels, pred_flat, average='weighted')  # Можно выбрать другой вариант усреднения: 'micro', 'macro', 'samples', 'weighted'
    return f1

In [22]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    elapsed_rounded = int(round((elapsed)))
    
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [23]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3060


In [24]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []

total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):

    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))

    t0 = time.time()
    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):

        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        res = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        loss = res['loss']
        logits = res['logits']

        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)            
    training_time = format_time(time.time() - t0)
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        with torch.no_grad():        

            res = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        loss = res['loss']
        logits = res['logits']

        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.3f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))
    training_stats.append(
        {
            
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

======== Epoch 1 / 2 ========
  Batch    40  of  5,542.    Elapsed: 0:01:20.
  Batch    80  of  5,542.    Elapsed: 0:02:37.
  Batch   120  of  5,542.    Elapsed: 0:03:54.
  Batch   160  of  5,542.    Elapsed: 0:05:11.
  Batch   200  of  5,542.    Elapsed: 0:06:28.
  Batch   240  of  5,542.    Elapsed: 0:07:45.
  Batch   280  of  5,542.    Elapsed: 0:09:02.
  Batch   320  of  5,542.    Elapsed: 0:10:19.
  Batch   360  of  5,542.    Elapsed: 0:11:37.
  Batch   400  of  5,542.    Elapsed: 0:12:54.
  Batch   440  of  5,542.    Elapsed: 0:14:11.
  Batch   480  of  5,542.    Elapsed: 0:15:28.
  Batch   520  of  5,542.    Elapsed: 0:16:58.
  Batch   560  of  5,542.    Elapsed: 0:18:15.
  Batch   600  of  5,542.    Elapsed: 0:19:33.
  Batch   640  of  5,542.    Elapsed: 0:20:50.
  Batch   680  of  5,542.    Elapsed: 0:22:07.
  Batch   720  of  5,542.    Elapsed: 0:23:24.
  Batch   760  of  5,542.    Elapsed: 0:24:41.
  Batch   800  of  5,542.    Elapsed: 0:25:58.
  Batch   840  of  5,542.    E

  Batch 1,400  of  5,542.    Elapsed: 0:44:58.
  Batch 1,440  of  5,542.    Elapsed: 0:46:15.
  Batch 1,480  of  5,542.    Elapsed: 0:47:32.
  Batch 1,520  of  5,542.    Elapsed: 0:48:49.
  Batch 1,560  of  5,542.    Elapsed: 0:50:07.
  Batch 1,600  of  5,542.    Elapsed: 0:51:24.
  Batch 1,640  of  5,542.    Elapsed: 0:52:41.
  Batch 1,680  of  5,542.    Elapsed: 0:53:58.
  Batch 1,720  of  5,542.    Elapsed: 0:55:15.
  Batch 1,760  of  5,542.    Elapsed: 0:56:32.
  Batch 1,800  of  5,542.    Elapsed: 0:57:49.
  Batch 1,840  of  5,542.    Elapsed: 0:59:06.
  Batch 1,880  of  5,542.    Elapsed: 1:00:24.
  Batch 1,920  of  5,542.    Elapsed: 1:02:42.
  Batch 1,960  of  5,542.    Elapsed: 1:05:09.
  Batch 2,000  of  5,542.    Elapsed: 1:06:42.
  Batch 2,040  of  5,542.    Elapsed: 1:08:32.
  Batch 2,080  of  5,542.    Elapsed: 1:09:53.
  Batch 2,120  of  5,542.    Elapsed: 1:12:58.
  Batch 2,160  of  5,542.    Elapsed: 1:14:21.
  Batch 2,200  of  5,542.    Elapsed: 1:16:09.
  Batch 2,240

In [25]:
df_stats = pd.DataFrame(data=training_stats)
df_stats = df_stats.set_index('epoch')
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,1.05149,1.046846,0.262112,2:58:17,0:26:55
2,1.04628,1.040827,0.262112,3:27:01,0:26:54
